<a href="https://colab.research.google.com/github/gershomrichardbruno/Ergonomics-Smart-Chair/blob/main/Ergonomics_Colab_WithPose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip uninstall -y mediapipe
!pip install mediapipe==0.10.5 opencv-python-headless --quiet


Found existing installation: mediapipe 0.10.21
Uninstalling mediapipe-0.10.21:
  Successfully uninstalled mediapipe-0.10.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [11]:
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]


Saving WhatsApp Video 2025-04-28 at 22.14.08_1dc4cea8.mp4 to WhatsApp Video 2025-04-28 at 22.14.08_1dc4cea8.mp4


In [12]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

cap = cv2.VideoCapture(video_path)
frames, angles, accelerations = [], [], []
prev_angle = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    angle = None
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        try:
            hip = np.array([landmarks[mp_pose.PoseLandmark.LEFT_HIP].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_HIP].y])
            knee = np.array([landmarks[mp_pose.PoseLandmark.LEFT_KNEE].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_KNEE].y])
            shoulder = np.array([landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y])
            v1 = hip - knee
            v2 = shoulder - hip
            cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
        except:
            pass

    if angle is not None:
        acceleration = abs(angle - prev_angle) if prev_angle is not None else 0
        prev_angle = angle
        angles.append(angle)
        accelerations.append(acceleration)
        frames.append((frame, results))

cap.release()


In [13]:
output_video = 'annotated_output.mp4'
height, width = 480, 640
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, 10, (width, height))  # slower fps

for i, (frame, results) in enumerate(frames):
    frame = cv2.resize(frame, (width, height))

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2))

    cv2.putText(frame, f"Angle: {angles[i]:.2f}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
    cv2.putText(frame, f"Accel: {accelerations[i]:.2f}", (10, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)

    out.write(frame)

out.release()


In [14]:
from google.colab import files
files.download('annotated_output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
# Uninstall conflicting libraries
!pip uninstall -y numpy mediapipe opencv-python-headless

# Reinstall the libraries
!pip install numpy mediapipe opencv-python-headless --quiet

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: mediapipe 0.10.5
Uninstalling mediapipe-0.10.5:
  Successfully uninstalled mediapipe-0.10.5
Found existing installation: opencv-python-headless 4.11.0.86
Uninstalling opencv-python-headless-4.11.0.86:
  Successfully uninstalled opencv-python-headless-4.11.0.86
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 4.25.8 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.8 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
